In [11]:
import requests
from bs4 import BeautifulSoup

def check_pep(person):
    if person=="":
      return False
    """
    Checks if an entity has search results on OpenSanctions.org.

    Args:
        entity_name (str): The name of the entity to search for.

    Returns:
        bool: True if search results are found, False otherwise.
    """
    person = person.replace(" ", "+")
    url = f"https://www.opensanctions.org/search/?scope=peps&q={person}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.content, "html.parser")
        print(url)
        # Check for the presence of search results (adjust the selector as needed)
        # This is a basic example, you might need to inspect the page
        # structure to find the most robust selector.
        results = soup.find_all("div", class_="Search_resultTitle__twair")
        results2 = soup.find_all("span", class_="badge bg-warning")
        check_flag=False
        for result in results2:
          if result.text=="Politician":
            check_flag= True
        return len(results) > 0 and check_flag

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return False

# def set_sanctioned_entity_val(e1,e2,intermediary):
#   if check_sanctioned_entity(e1):
#     obj.sanct_entity_e1=1
#   if check_sanctioned_entity(e2):
#     obj.sanct_entity_e2=1
#   if check_sanctioned_entity(intermediary):
#     obj.sanct_entity_intermediary=1

print(check_pep("Rahul Gandhi"))

https://www.opensanctions.org/search/?scope=peps&q=Rahul+Gandhi
True


### THIS NOTEBOOK MUST BE RUN AND THE PUBLIC IP MUST BE COPIED AND ADDED IN THE riskScore.py file 
### Had to do this as we were'nt able to scrape it realtime locally, but somehow seems to be working in Google Collab
### SO this is a flask app just to deduce whether entities are sanctioned or persons are PEPs 

In [12]:
persons=["Rahul Gandhi","Arnav Banerjee"]
def persons_norm(persons):
    """Normalize PEP risk for a list of persons.

    Args:
        persons: List of person names

    Returns:
        float: Normalized PEP risk score
    """
    if not persons:
        return 0

    var = 0
    for person in persons:
        var = var + (1 if check_pep(person) else 0)
    return str(var / len(persons))
persons_norm(persons)

https://www.opensanctions.org/search/?scope=peps&q=Rahul+Gandhi
https://www.opensanctions.org/search/?scope=peps&q=Arnav+Banerjee


'0.5'

In [13]:
def check_sanctioned_entity(entity_name):
    if entity_name=="":
      return False
    """
    Checks if an entity has search results on OpenSanctions.org.

    Args:
        entity_name (str): The name of the entity to search for.

    Returns:
        bool: True if search results are found, False otherwise.
    """
    entity_name = entity_name.replace(" ", "+")
    url = f"https://www.opensanctions.org/search/?scope=securities&q={entity_name}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.content, "html.parser")

        # Check for the presence of search results (adjust the selector as needed)
        # This is a basic example, you might need to inspect the page
        # structure to find the most robust selector.
        results = soup.find_all("div", class_="Search_resultTitle__twair")
        results2 = soup.find_all("span", class_="badge bg-warning")
        check_flag=False
        for result in results2:
          if result.text=="Sanctioned entity":
            check_flag= True
        return len(results) > 0 and check_flag

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return False

In [14]:
def entities_norm(entities):
  var=0
  for entity in entities:
    var=var+check_sanctioned_entity(entity)
  return str(var/len(entities))


In [ ]:
!pip install flask ngrok
!pip install pyngrok
from flask import Flask, jsonify,request
from pyngrok import ngrok
import requests
ngrok.set_auth_token("2uqw1cjUHjzVoKjNTmjBH1yGRUl_5KZWyWLP7xgg4EnUjtDYj") # Replace YOUR_AUTHTOKEN with your actual token

app = Flask(__name__)

@app.route('/data')
def get_data():
    data = {'message': 'Hello from Colab!'}
    return jsonify(data)

@app.route('/persons', methods=['GET'])
def get_strings():
    """
    This endpoint accepts a JSON payload containing a list of strings.

    Example JSON payload:
    {
        "strings": ["string1", "string2", "string3"]
    }

    Returns a JSON response containing the received strings.
    """
    try:
        data = request.get_json()  # Get the JSON data from the request
        strings = data.get('strings', [])  # Extract the list of strings
        return persons_norm(strings)
    except Exception as e:
        return jsonify({'error': str(e)}),

@app.route('/entities', methods=['GET'])
def get_entityScore():
    """
    This endpoint accepts a JSON payload containing a list of strings.

    Example JSON payload:
    {
        "strings": ["string1", "string2", "string3"]
    }

    Returns a JSON response containing the received strings.
    """
    try:
        data = request.get_json()  # Get the JSON data from the request
        strings = data.get('strings', [])  # Extract the list of strings
        return entities_norm(strings)
    except Exception as e:
        return jsonify({'error': str(e)}),

# Open a ngrok tunnel to the Flask app
ngrok_tunnel = ngrok.connect(5000)
print('Public URL:', ngrok_tunnel.public_url)

# Run the Flask app
app.run(port=5000)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.0 MB/s eta 0:00:00
Public URL: https://227c-34-139-148-219.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 12:05:59] "GET /entities HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 12:11:02] "POST /persons HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 12:11:03] "POST /persons HTTP/1.1" 405 -


https://www.opensanctions.org/search/?scope=peps&q=Maria+Gonzalez


INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 12:11:44] "GET /persons HTTP/1.1" 200 -


https://www.opensanctions.org/search/?scope=peps&q=Rahul+Gandhi
https://www.opensanctions.org/search/?scope=peps&q=Maria+Gonzalez


...

INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 12:15:16] "GET /entities HTTP/1.1" 200 -


https://www.opensanctions.org/search/?scope=peps&q=Maria+Gonzalez


INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 12:15:17] "GET /persons HTTP/1.1" 200 -


https://www.opensanctions.org/search/?scope=peps&q=Rahul+Gandhi


INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 12:15:19] "GET /entities HTTP/1.1" 200 -
